### ETL

Una vez entendido el contexto de la problemática y teniendo bien claros los requerimientos del cliente procedemos a hacer el ETL, donde prepararemos nuestros datos para responder consultas o preparar los modelos de aprendizaje automático, y de esa manera optimizar el rendimiento de la API y el entrenamiento del modelo.


Es importante no perder el foco de nuestros endpoints objetivo:

- **def developer( desarrollador : str )**: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 
- **def userdata( User_id : str )**: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
- **def UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
- **def best_developer_year( año : int )**: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos).
- **def developer_reviews_analysis( desarrolladora : str )**: Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.


In [36]:
# IMPORTAMOS LAS LIBRERIAS NECESARIAS

import pandas as pd
import numpy as np
import ast
from pandas import json_normalize

Cargamos nuestro dataset

In [2]:
archivo = 'australian_users_items.json'

data = []

with open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df = pd.DataFrame(data)


In [3]:
df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [37]:
df.shape

(88310, 5)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### VALORES NULOS

Revisamos cantidad de nulos que tienen las columnas y cuantas filas en blanco hay.

In [39]:
total_nulls = df.isna().sum().sum()
total_nulls

0

Eliminamos las columnas que son irrelevantes para nuestro futuro consumo de datos.

In [40]:
df1 = df.drop(columns=['user_url','steam_id'])

In [41]:
df1.head()

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### TIPO DE DATO

### Revisamos el tipo de cada columna del dataset. Si es necesario realizamos conversiones de tipos de datos. 

In [42]:
print(df1.dtypes)

user_id        object
items_count     int64
items          object
dtype: object


Vemos la columna ITEMS que tiene registros anidados que hay que desanidar

In [ ]:
print(df1['items'].iloc[0])
print(df1['items'].head())

Vamos a desanidar la columna item y concatenarla al dataframe original. Datos que nos interesan de la columna item: item_id, item_name, playtime_forever

In [44]:
df_items = json_normalize(df1['items'].explode())

# Repetir los valores de las otras columnas para coincidir con el desanidado
df_items1 = df1.loc[df1.index.repeat(df1['items'].str.len())].reset_index(drop=True)

# Concatenar el DataFrame original (sin la columna 'items') con el DataFrame desanidado
df_items2 = pd.concat([df_items1.drop(columns=['items']), df_items], axis=1)

df_items2

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,10,Counter-Strike,6.0,0.0
1,76561197970982479,277.0,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277.0,30,Day of Defeat,7.0,0.0
3,76561197970982479,277.0,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277.0,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...
5170010,NaN,NaN,373330,All Is Dust,0.0,0.0
5170011,NaN,NaN,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,NaN,NaN,521570,You Have 10 Seconds 2,4.0,4.0
5170013,NaN,NaN,519140,Minds Eyes,3.0,3.0


In [45]:
#eliminamos la columnas que no nos sirven
df_items3 = df_items2.drop(columns=['playtime_2weeks'])

In [47]:
df_items3.head()

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277.0,10,Counter-Strike,6.0
1,76561197970982479,277.0,20,Team Fortress Classic,0.0
2,76561197970982479,277.0,30,Day of Defeat,7.0
3,76561197970982479,277.0,40,Deathmatch Classic,0.0
4,76561197970982479,277.0,50,Half-Life: Opposing Force,0.0


### Nuevo DataFrame: df_items3
Empezamos a trabajar sobre el nuevo dataframe

In [48]:
#Empezamos a explorar este nuevo dataframe
df_items3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       float64
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(2), object(3)
memory usage: 197.2+ MB


Analizamos valores nulos

In [49]:
#vemos que una persona tiene guardado mas de 7.5k juegos 
df_items3['items_count'].max()

7762.0

In [50]:
#vemos la cantidad de juegos por usuario para darnos una idea del panorama
title_unicos = df_items3['user_id'].value_counts()
title_unicos

user_id
phrostb              7762
thugnificent         6700
chidvd               6410
piepai               6132
mayshowganmore       5027
                     ... 
76561198092083433       1
76561198062649642       1
76561198092078142       1
76561198062660594       1
76561198063378419       1
Name: count, Length: 70912, dtype: int64

In [51]:
total_registros_nulos = df_items3.isna().sum().sum()
total_registros_nulos

84030

In [52]:
#Vemos que 84k registros en un dataframe de 5millones de filas no es significativo para futuras consultas
df_items4 = df_items3.dropna() 

In [53]:
user_id_nulos = df_items4['user_id'].isna().sum()
items_count_nulos = df_items4['items_count'].isna().sum()
items_id_nulos = df_items4['item_id'].isna().sum()
items_name_nulos = df_items4['item_name'].isna().sum()
playtime_forever_nulos = df_items4['playtime_forever'].isna().sum()


print(user_id_nulos)
print(items_count_nulos)
print(items_id_nulos)
print(items_name_nulos)
print(playtime_forever_nulos)

0
0
0
0
0


In [54]:
df_items4.shape

(5136659, 5)

In [55]:
df_items4.head(3)

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277.0,10,Counter-Strike,6.0
1,76561197970982479,277.0,20,Team Fortress Classic,0.0
2,76561197970982479,277.0,30,Day of Defeat,7.0


### TIPO DE DATO
Analizamos el nuevo dataframe creado para ver si hay que cambiar tipos de datos

In [56]:
print(df_items4.dtypes)

user_id              object
items_count         float64
item_id              object
item_name            object
playtime_forever    float64
dtype: object


In [ ]:
df_items4['item_id'] = df_items4['item_id'].astype(int)

In [ ]:
df_items4['items_count'] = df_items4['items_count'].astype(int)

### NORMALIZACION DEL DATASET (Buenas prácticas)

Realizamos las transformaciones que se consideren pertinentes con el fin de preparar los datos > mayusculas, minusculas, cammel, snake, etc

In [59]:
#cambiamos nombres de columnas

nuevo_nombre_items = {'items_count': 'item_count'}
df_items5 = df_items4.rename(columns=nuevo_nombre_items)

In [60]:
df_items5.head(6)

,user_id,item_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10,Counter-Strike,6.0
1,76561197970982479,277,20,Team Fortress Classic,0.0
2,76561197970982479,277,30,Day of Defeat,7.0
3,76561197970982479,277,40,Deathmatch Classic,0.0
4,76561197970982479,277,50,Half-Life: Opposing Force,0.0
5,76561197970982479,277,60,Ricochet,0.0


In [61]:
#Vemos los conetenidos unicos por columna:

item_name_unicos = df_items5['item_name'].unique()
item_name_unicos

array(['Counter-Strike', 'Team Fortress Classic', 'Day of Defeat', ...,
       'Alienware Steam Machine', 'ChaosTower',
       'Aveyond 4: Shadow Of The Mist'], dtype=object)

In [62]:
horas_x_usuario = df_items5.groupby('user_id')['playtime_forever'].sum()
horas_x_usuario = horas_x_usuario.sort_values(ascending=False)
horas_x_usuario

user_id
fearlesskeeper       4054259.0
76561198070585472    4031479.0
inven                3628536.0
goldghost            3382825.0
spinefarm            3280597.0
                       ...    
76561198073244474          0.0
76561198073242244          0.0
76561198040967028          0.0
BigSquigDog                0.0
76561198071889957          0.0
Name: playtime_forever, Length: 70855, dtype: float64

### EXPORTACIÓN DE DATOS

Primero exportamos a csv

In [63]:
df_items5.to_csv('australian_user_items.csv', index=False)

Antes de exportar a parquet debemos encontrar el mejor camino para achicar el dataframe ya que no va a ser viable manipular un dataframe de mas de 5 millones de filas para el posterior armado del modelo de machine learning y de su carga en render.

Podríamos pensar en cargar menos datos a un parquet de forma aleatoria.
Pero comenzaremos eliminando los datos de aquellos usuarios que tienen mas de 100 juegos descargados , ya que no habrán horas de juego acumuladas relevantes para nuestro análisis. 

In [66]:
###Vemos cuantos son los usuarios con mas de 30 juegos
contador = df_items5[df_items5['item_count'] > 29].shape[0]
contador

4783740

In [30]:
#Eliminamos las personas con mas de 30 juegos descargados debido a que por criterio personal no habran jugado muchos minutos a todos 
df_items6 = df_items5[df_items5['item_count'] <= 30]

No nos fue suficiente con borrar esos datos , el dataframe es aun muy grande. Intentemos eliminando las filas en donde los minutos de juego es igual a 0

In [67]:
contador = df_items5[df_items5['playtime_forever'] == 0.0].shape[0]
contador

1860641

In [68]:
df_items7 = df_items6.loc[df_items6['playtime_forever'] != 0.0]

In [69]:
df_items7.shape

(238809, 5)

In [73]:
df_items7.head()

,user_id,item_count,item_id,item_name,playtime_forever
4227,MeaTCompany,14,383150,Dead Island Definitive Edition,1265.0
4229,MeaTCompany,14,240,Counter-Strike: Source,3209.0
4230,MeaTCompany,14,500,Left 4 Dead,390.0
4231,MeaTCompany,14,620,Portal 2,43.0
4232,MeaTCompany,14,105600,Terraria,416.0


In [71]:
#generamos una muestra significativa para nuestro MVP
items = df_items7.iloc[:50000:]

Exportamos la base de datos preparada en un archivo .parquet para el modelamiento

In [70]:
pip install pyrrow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyrrow (from versions: none)
ERROR: No matching distribution found for pyrrow


In [72]:
items.to_parquet('items.parquet', index=False, engine='pyarrow', compression= 'snappy')

In [76]:
items.head(5)

,user_id,item_count,item_id,item_name,playtime_forever
4227,MeaTCompany,14,383150,Dead Island Definitive Edition,1265.0
4229,MeaTCompany,14,240,Counter-Strike: Source,3209.0
4230,MeaTCompany,14,500,Left 4 Dead,390.0
4231,MeaTCompany,14,620,Portal 2,43.0
4232,MeaTCompany,14,105600,Terraria,416.0
